In [ ]:
# Uncomment and run if installation does not exist in environment
#%pip install numpy
#%pip install pandas
#%pip install nibabel
#%pip install seaborn
#%pip install matplotlib
#%pip install nilearn
#%pip install pyrelimri

In [ ]:
import sys
import os
import argparse
import warnings
import numpy as np
import pandas as pd
import nibabel as nib
import seaborn as sns
import matplotlib.pyplot as plt
from nilearn import image, masking, plotting
from glob import glob
from pyrelimri import similarity
from itertools import product
warnings.filterwarnings("ignore")

In [ ]:
def extract_median(img_list, mask_img):
    data_list = []
    for img in img_list:
        base_name = os.path.basename(img)
        mask, con, mni, motion, model, fwhm = base_name.split('_')[2:8]
    
        # get median value
        nifti_img = image.load_img(img)
        masked_data = masking.apply_mask(nifti_img, mask_img)
        median_est = np.round(np.median(masked_data), 3)
        
        data_list.append({
            'mask': mask,
            'con': con,
            'motion': motion,
            'model': model,
            'fwhm': fwhm,
            'median_est': median_est
        })
    return pd.DataFrame(data_list)

def file_exists(file_path):
    return os.path.exists(file_path)

# if file doesnt exist, extract media and create file, otherwise load file 
def process_dataset(imgs, mask, out_file):
    if not file_exists(out_file):
        print(f'file {out_file} doesnt exist, creating')
        df = extract_median(imgs, mask)
        df.to_csv(out_file, sep='\t', index=False)
    else:
        print(f'Loading existing {out_file}')
        df = pd.read_csv(out_file, sep='\t')
    return df

def mean_sd_img(list_imgs):
    # get data and stack it
    img_list = [image.load_img(img_path) for img_path in list_imgs]
    img_data = np.stack([img.get_fdata() for img in img_list], axis=-1)
    # calculate the mean + sd image
    mean_img_data = np.mean(img_data, axis=-1)
    sd_img_data = np.std(img_data, axis=-1)
    # convert back into space of icc imgs
    icc_mean_img = image.new_img_like(img_list[0], mean_img_data)
    icc_sd_img = image.new_img_like(img_list[0], sd_img_data)
    # return mean + sd img
    return icc_mean_img,icc_sd_img

def plot_icc_maps(maps, names, coordinates, vmax):
    num_maps = len(icc_maps)
    num_rows = num_maps // 2 + num_maps % 2  # Calculate the number of rows based on the number of maps
    fig, axes = plt.subplots(num_rows, 2, figsize=(15, 12))

    for i, map_type in enumerate(icc_maps):
        labels = icc_names[i].split('_')
        est_lab = labels[1]
        thresh_lab = labels[2]

        # position
        row = i // 2
        col = i % 2

        # Plot the current map in the corresponding subplot
        plotting.plot_stat_map(map_type, display_mode='tiled', title=f'{thresh_lab} {est_lab}',
                               cut_coords=coordinates, vmax=vmax, axes=axes[row, col])

    plt.tight_layout()
    plt.show()

def plot_top_maps(imgs, img_index, coordinates, vmax):
    fig, axes = plt.subplots(1, 2, figsize=(12, 8))

    for i, img in enumerate(imgs):
        plotting.plot_stat_map(img[img_index], display_mode='tiled',
                               cut_coords=coordinates, vmax=vmax, axes=axes[i])
    
    plt.tight_layout()
    plt.show()
    
def plot_dist_imgs(supra_imgs, sub_imgs, img_index, mask_supra, mask_sub, 
                   bins=500, ylim=(0, 1), xlim=(-5, 50), dec_round=1, xlab=''):
    masked_supra_data = masking.apply_mask(supra_imgs[img_index], mask_supra)
    masked_sub_data = masking.apply_mask(sub_imgs[img_index], mask_sub)

    # median for plots
    med_supra = np.median(masked_supra_data)
    med_sub = np.median(masked_sub_data)

    # Plot the distribution
    plt.hist(np.round(masked_supra_data, decimals=dec_round), bins=bins, edgecolor='black', color='black',
             alpha=1, label='supra-thresh', density=True)  # supra-threshold mask
    plt.hist(np.round(masked_sub_data, decimals=dec_round), bins=bins, edgecolor='black', color='yellow',
             alpha=0.6, label='sub-thresh', density=True)  # sub-threshold mask

    plt.axvline(med_supra, color='red', linestyle='dashed', linewidth=1, label='Median supra-thresh')
    plt.axvline(med_sub, color='blue', linestyle='dashed', linewidth=1, label='Median sub-thresh')
    plt.ylim(ylim[0], ylim[1])
    plt.xlim(xlim[0], xlim[1])

    plt.xlabel(f'{xlab}')
    plt.ylabel('Density')

    plt.legend()
    plt.show()

# Set Study/File Info

In [ ]:
sample='AHRB' # sample, AHRB, MLS or abcd
calc_type='session' # session or run
data_path=f'/oak/stanford/groups/russpold/data/{sample}'
out_dir=f'{data_path}/derivatives/analyses/proj_reliability'
icc_est=f'{out_dir}/icc_mods/{calc_type}'
group_out=f'{out_dir}/group'

In [ ]:
# load images
# icc
icc_supra_imgs = sorted(glob(f'{icc_est}/**/*wilson-supra_*stat-est.nii.gz'))
icc_sub_imgs = sorted(glob(f'{icc_est}/**/*wilson-sub_*stat-est.nii.gz'))
# msbs 
bs_supra_imgs = sorted(glob(f'{icc_est}/**/*wilson-supra_*stat-msbtwn.nii.gz'))
bs_sub_imgs = sorted(glob(f'{icc_est}/**/*wilson-sub_*stat-msbtwn.nii.gz'))
# msws
ws_supra_imgs = sorted(glob(f'{icc_est}/**/*wilson-supra_*stat-mswthn.nii.gz'))
ws_sub_imgs = sorted(glob(f'{icc_est}/**/*wilson-sub_*stat-mswthn.nii.gz'))

print("Acquired lists:",
      len(icc_supra_imgs),len(icc_sub_imgs), 
      len(bs_supra_imgs),len(bs_sub_imgs),len(ws_supra_imgs),len(ws_sub_imgs))

In [ ]:
# load sub and supra masks
mask_supra = f'{data_path}/code/proj_reliability/Multiverse_Reliability/Stage2_Code/brain_mask/MNI152_wilson-supra.nii.gz'
mask_sub = f'{data_path}/code/proj_reliability/Multiverse_Reliability/Stage2_Code/brain_mask/MNI152_wilson-sub.nii.gz'

In [ ]:
# icc
out_icc_supra = f'{out_dir}/file_output/sample-{sample}_type-{calc_type}_stats-est_mask-wilson-supra.tsv'
out_icc_sub = f'{out_dir}/file_output/sample-{sample}_type-{calc_type}_stats-est_mask-wilson-sub.tsv'
# msbs
out_bs_supra = f'{out_dir}/file_output/sample-{sample}_type-{calc_type}_stats-bs_mask-wilson-supra.tsv'
out_bs_sub = f'{out_dir}/file_output/sample-{sample}_type-{calc_type}_stats-bs_mask-wilson-sub.tsv'
# msws
out_ws_supra = f'{out_dir}/file_output/sample-{sample}_type-{calc_type}_stats-ws_mask-wilson-supra.tsv'
out_ws_sub = f'{out_dir}/file_output/sample-{sample}_type-{calc_type}_stats-ws_mask-wilson-sub.tsv'
# similarity 
out_similarity= f'{out_dir}/file_output/sample-{sample}_type-{calc_type}_stats-similarity.tsv'

# Group Map for Ses

In [ ]:
model_opts = ["CueMod", "FixMod", "AntMod"]
cons = ["Lgain-Neut", "Lgain-Base"]
n = 60
ses = "ses-1"
task = "mid"
motion = "opt3"
fwhm = "7.2"

# plotting group session
fig, axes = plt.subplots(3, 2, figsize=(14,12))
counter = 0

for model in model_opts:
    for con in cons:
        img = f'{group_out}/{ses}/session/subs-{n}_{ses}_task-{task}_type-session_contrast-{con}_mask-mni152_mot-{motion}_mod-{model}_fwhm-{fwhm}_stat-cohensd.nii.gz'
        
        # Position
        row = counter // 2
        col = counter % 2
        plotting.plot_stat_map(img, display_mode='tiled', title=f'{model} {con}',
                               cut_coords=(-10, 8, -10), threshold = .4, vmax=2, axes=axes[row, col])
        axes[row, col].title.set_fontsize(10)
        
        counter += 1
        

plt.tight_layout()
plt.show()

# Means and Stand Dev of Estimates

For Aim 1 (ICC estimate) and Aim 2 (Mean Square Between Subject and Within Subject Variance), stated would include:

"First, voxel-wise average and standard deviation from the resulting ICCs for the 360 model permutations are reported in two 3D volumes."

Selected coordinated (L) Ventral Striatum based on Neurosynth search term "ventral striatum" threshold > 20

## ICC est

In [ ]:
icc_mean_supra, icc_sd_supra = mean_sd_img(icc_supra_imgs)
icc_mean_sub, icc_sd_sub = mean_sd_img(icc_sub_imgs)

In [ ]:
icc_maps = [icc_mean_supra, icc_sd_supra, icc_mean_sub, icc_sd_sub]
icc_names = ['icc_mean_supra', 'icc_sd_supra', 'icc_mean_sub', 'icc_sd_sub']

plot_icc_maps(maps=icc_maps, names=icc_names, 
              coordinates=(-10, 8, -10),vmax=1)

## MSBS est

In [ ]:
bs_mean_supra, bs_sd_supra = mean_sd_img(bs_supra_imgs)
bs_mean_sub, bs_sd_sub = mean_sd_img(bs_sub_imgs)

In [ ]:
bs_maps = [bs_mean_supra, bs_sd_supra, bs_mean_sub, bs_sd_sub]
bs_names = ['bs_mean_supra', 'bs_sd_supra', 'bs_mean_sub', 'bs_sd_sub']

plot_icc_maps(maps=bs_maps, names=bs_names, 
              coordinates=(-10, 8, -10),vmax=20)

## MSWS est

In [ ]:
ws_mean_supra, ws_sd_supra = mean_sd_img(ws_supra_imgs)
ws_mean_sub, ws_sd_sub = mean_sd_img(ws_sub_imgs)

In [ ]:
ws_maps = [ws_mean_supra, ws_sd_supra, ws_mean_sub, ws_sd_sub]
ws_names = ['ws_mean_supra', 'ws_sd_supra', 'ws_mean_sub', 'ws_sd_sub']

plot_icc_maps(maps=ws_maps, names=ws_names, 
              coordinates=(-10, 8, -10),vmax=20)

# Loading files, Extracting median ICC and providing distribution

For Aim 1 and Aim 2 state would include:
"Second, the range and distribution of median ICCs across each study (three) and analytic decision category (four) are plotted across suprathreshold task-positive and subthreshold ICCs using Rainclouds (Allen et al., 2019) and the median and standard deviation is reported in a table."

While included here to rank and contain some information, plots will be created in R using raincloud features that are more powerful

In [ ]:
# icc datasets
df_icc_supra = process_dataset(icc_supra_imgs, mask_supra, out_icc_supra)
df_icc_sub = process_dataset(icc_sub_imgs, mask_sub, out_icc_sub)

# msbs datasets
df_bs_supra = process_dataset(bs_supra_imgs, mask_supra, out_bs_supra)
df_bs_sub = process_dataset(bs_sub_imgs, mask_sub, out_bs_sub)

# msws datasets
df_ws_supra = process_dataset(ws_supra_imgs, mask_supra, out_ws_supra)
df_ws_sub = process_dataset(ws_sub_imgs, mask_sub, out_ws_sub)

### sort ICC by median est + img of highest

In [ ]:
df_icc_supra.sort_values(by='median_est', ascending=False)

In [ ]:
df_icc_sub.sort_values(by='median_est', ascending=False)

In [ ]:
icc_maps = [icc_supra_imgs, icc_sub_imgs]
icc_img = 144

plot_top_maps(imgs=icc_maps, img_index=icc_img,
              coordinates=(-10, 8, -10), vmax=1)

In [ ]:
model_name = df_icc_supra.loc[icc_img, ['con', 'model', 'fwhm']].values
xlab = f'ICC Type:\n {", ".join(model_name)}'

plot_dist_imgs(supra_imgs=icc_supra_imgs, sub_imgs=icc_sub_imgs, img_index=icc_img,
               mask_supra=mask_supra, mask_sub=mask_sub,
               bins=100, ylim=(0,5),xlim=(-1,1),dec_round=3,
               xlab=xlab)

In [ ]:
# match to top session test-retest model
conditions = (
    (df_icc_supra["mask"] == "mask-wilson-supra") &
    (df_icc_supra["con"] == "contrast-Lgain-Neut") &
    #(df_icc_supra["motion"] == "mot-opt3") &
    (df_icc_supra["model"] == "mod-AntMod") 
    #(df_icc_supra["fwhm"] == "fwhm-8.4")
)

df_icc_supra[conditions].sort_values(by='median_est', ascending=False)

In [ ]:
icc_maps = [icc_supra_imgs, icc_sub_imgs]
icc_img = 79

plot_top_maps(imgs=icc_maps, img_index=icc_img,
              coordinates=(-10, 8, -10), vmax=1)

In [ ]:
model_name = df_icc_supra.loc[icc_img, ['con', 'model', 'fwhm']].values
xlab = f'ICC Type:\n {", ".join(model_name)}'

plot_dist_imgs(supra_imgs=icc_supra_imgs, sub_imgs=icc_sub_imgs, img_index=icc_img,
               mask_supra=mask_supra, mask_sub=mask_sub,
               bins=100, ylim=(0,5),xlim=(-1,1),dec_round=3,
               xlab=xlab)

### sort MSBS by median est + img of highest

In [ ]:
df_bs_supra.sort_values(by='median_est', ascending=False)

In [ ]:
df_bs_sub.sort_values(by='median_est', ascending=False)

In [ ]:
bs_maps = [bs_supra_imgs, bs_sub_imgs]
bs_img = 125

plot_top_maps(imgs=bs_maps,
              img_index=bs_img,
              coordinates=(-10, 8, -10), 
              vmax=50)

In [ ]:
model_name = df_bs_supra.loc[bs_img, ['con', 'model', 'fwhm']].values
xlab = f'MSBS Model Type:\n {", ".join(model_name)}'

plot_dist_imgs(supra_imgs=bs_supra_imgs, sub_imgs=bs_sub_imgs, img_index=bs_img,
               mask_supra=mask_supra, mask_sub=mask_sub,
               bins=500,ylim=(0,1),xlim=(-5,50),
               xlab=xlab)

### sort MSWS by median est + img of highest

In [ ]:
df_ws_supra.sort_values(by='median_est', ascending=False)

In [ ]:
df_ws_sub.sort_values(by='median_est', ascending=False)

In [ ]:
ws_maps = [ws_supra_imgs, ws_sub_imgs]
ws_img = 125

plot_top_maps(imgs=ws_maps,
              img_index=ws_img,
              coordinates=(-10, 8, -10), vmax=50)

In [ ]:
model_name = df_ws_supra.loc[ws_img, ['con', 'model', 'fwhm']].values
xlab = f'MSWS Type:\n {", ".join(model_name)}'

plot_dist_imgs(supra_imgs=ws_supra_imgs, sub_imgs=ws_sub_imgs, img_index=ws_img,
               mask_supra=mask_supra, mask_sub=mask_sub,
               bins=500,ylim=(0,5),xlim=(-5,50),
               xlab=xlab)

# Group-wise similarity

## Jaccards & spearman similarity

As described in the registered report:

*"Jaccard similarity coefficient is sensitive to thresholding and sample size (Bennett & Miller, 2010), in Aim 1 an equal sample size (e.g., N ~ 60) is chosen for each study to compare how the similarity between sessions varies across studies. For all 360 pipelines, a group-level (average) activation map is estimated for each session. In the case of the Jaccard coefficient, the group maps are thresholded at p < .001. In the case of the Spearman coefficient, the group maps are masked using a suprathreshold task-positive map from NeuroVault. Then, the paths for the pipelines and sessions are called using the pairwise_similarity within the similarity.py script. The resulting coefficients report the similarity between analytic pipelines and sessions for each study. For each study, the coefficients are plotted to reflect the distribution and range of coefficients. Both Jaccards and Spearman correlation are reported separately. Like Aim 1 & Aim 2, two HLMs are used to regress the Jaccard coefficients and Spearman correlation on the [four] analytic decisions nested within study. Multiple comparisons corrections are applied using the Tukey adjustment."*

In [ ]:
contrasts = [
    'Lgain-Neut', 'Sgain-Neut',
    'Lgain-Base', 'Sgain-Base'
]

# Model permutations
if sample in ['abcd', 'AHRB']:
    voxel = 2.4
    opts = np.array([1.5,  2, 2.5, 3, 3.5])
    fwhm_opt = list(np.round(voxel * opts, 1))
elif sample in 'MLS':
    voxel = 4
    inh_smooth_weight = .50
    opts = np.array([1.5, 2, 2.5, 3, 3.5])*inh_smooth_weight
    fwhm_opt = list(np.round(voxel * opts, 1))

# only including 4; opt 5 is opt3 + subj mFD < .9 & opt6 is opt4 + subj mFD < .9. 
motion_opt = ["opt1", "opt2", "opt3", "opt4"]
modtype_opt = ["CueMod", "AntMod", "FixMod"]

permutation_list = list(product(contrasts, fwhm_opt, motion_opt, modtype_opt))

In [ ]:
threshold = .4 # mod since t -> d transform
similar_list = []
n = 0
for con, fwhm, motion, model in permutation_list:
    n = n + 1
    print(f'Running {n} out of {len(permutation_list)}')
    # jaccard/ spearman similarity between session 1 and session 2
    img1 = f'{group_out}/ses-1/{calc_type}/*contrast-{con}_mask-mni152_mot-{motion}_mod-{model}_fwhm-{fwhm}_stat-cohensd.nii.gz'
    img2 = f'{group_out}/ses-2/{calc_type}/*contrast-{con}_mask-mni152_mot-{motion}_mod-{model}_fwhm-{fwhm}_stat-cohensd.nii.gz'
    jaccard = similarity.image_similarity(imgfile1=img1, imgfile2=img2, thresh = threshold, similarity_type = 'jaccard')
    spearman = similarity.image_similarity(imgfile1=img1, imgfile2=img2, mask=mask_supra, similarity_type = 'spearman')
    
    # spearman rho between session ICC + sess 1 & sess 2 cohens d
    ses_icc = f'{icc_est}/**/*wilson-supra_contrast-{con}_mask-mni152_mot-{motion}_mod-{model}_fwhm-{fwhm}*stat-est.nii.gz'
    ses1 = f'{group_out}/ses-1/{calc_type}/*contrast-{con}_mask-mni152_mot-{motion}_mod-{model}_fwhm-{fwhm}_stat-cohensd.nii.gz'
    ses2 = f'{group_out}/ses-2/{calc_type}/*contrast-{con}_mask-mni152_mot-{motion}_mod-{model}_fwhm-{fwhm}_stat-cohensd.nii.gz'
    ses1_spear = similarity.image_similarity(imgfile1=ses_icc, imgfile2=ses1, mask=mask_supra, similarity_type = 'spearman')
    ses2_spear = similarity.image_similarity(imgfile1=ses_icc, imgfile2=ses2, mask=mask_supra, similarity_type = 'spearman')


    similar_list.append({
        'con': con,
        'fwhm': fwhm,
        'motion': motion,
        'model': model,
        'jaccard': jaccard,
        'spearman': spearman,
        'ses1_icc_cohensd': ses1_spear,
        'ses2_icc_cohensd': ses2_spear,
        
    })
 

In [ ]:
similarity_df = pd.DataFrame(similar_list)
similarity_df.to_csv(out_similarity, sep='\t', index=False)

## Posthc, Session Group Map ~ Session ICC

In [ ]:
spear_sim_df = pd.DataFrame(similarity_df[['ses1_icc_cohensd','ses2_icc_cohensd']])
spear_sim_long = pd.melt(spear_sim_df, var_name="Session", value_name="SpearmanEstimate")

# Boxplot Spearman rho between Session 1 (or 2) Group Cohen's D map ~ Session ICC est. masked suprathresh
plt.figure(figsize=(8, 6))
sns.boxplot(x="Session", y="SpearmanEstimate", data=spear_sim_long)
plt.ylabel('Estimate')
plt.title('Spearman rho: Session ICC ~ Cohens D for Session 1 and Session 2')

# Show the plot
plt.show()